In [121]:
import numpy as np
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb
import combat_probability as cp
from icepool import d, Die

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = True
SAVETABS = True

COLOR_LIST = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [122]:
# functions for generating alternate tables
import pandas as pd

def convert_die(n, p_enc, n0=1, n1=None):
    """Creates a "rolls per encounter" table for a n-sided die from a conventional random encounter table.
    """
    delta = 1.0/n
    p_non = 1 - p_enc
    if n1:
        w = 1 - p_non**(n1 - (n0 - 1))
    else:
        w = 1

    days_est = []
    die = {}
    for i in range(n):
        cdf = (i+0.5)*delta
        d_est = np.ceil((n0 - 1) + (np.log(1 - cdf*w)/np.log(p_non)))

        #d_est = np.ceil(np.log(1 - cdf)/np.log(p_non))
        #d_est = max(1, np.round(np.log(1 - cdf)/np.log(1 - p_enc), 0))
        days_est += [d_est]
        die[d_est] = die.get(d_est, 0) + 1
    
    return die

def construct_dataframe(d_dict):
    roll_values = []
    d = Die(d_dict)
    d_min = 0
    d_max = 0
    for k, v in d.items():
        d_min = d_max + 1
        d_max = d_max + v
        if d_min == d_max:
            roll_values += [f'{d_min}']
        else:
            roll_values += [f'{d_min}-{d_max}']

    roll_name = f'1d{d_max}'
    d = {
        roll_name: roll_values, 
        'Days': [int(k) for k in d_dict.keys()],
    }
    df = pd.DataFrame(data=d)
    return df

def print_die_dict(d_dict):
    d = Die(d_dict)
    d_min = 0
    d_max = 0
    for k, v in d.items():
        d_min = d_max + 1
        d_max = d_max + v
        if d_min == d_max:
            print(f'| {d_min} | {k:.0f} |')
        else:
            print(f'| {d_min}-{d_max} | {k:.0f} |')

In [123]:
# Fig 1: Shows how CV for the number of encounters depends on the probability of a single roll resulting in an encounter.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter probability',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='CV (sigma / mean)',
            range=[0, 3],
            tickformat='.1f',
            tick0=0, dtick=0.5,
            #minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

p_enc = np.linspace(0.01, 1, 100)
cv = np.sqrt( (1 - p_enc)/(p_enc))


fig.add_trace(go.Scatter(
    x=p_enc,
    y=cv,
    name=f'encounter probability',
    mode='lines',
    hovertemplate=
        'encounter probability %{x:.0%}<br>'+
        'sigma / mean %{y:.2f}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-cv-scaling-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-cv-scaling-small')

In [124]:
# Fig 2: Shows the probability distribution for the number of encounters might occur after eight rolls
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounters',
            range=[-1, 8],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,0.41],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.02),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

p_enc = 0.3
p_non = 0.7
d = Die({0: 7, 1: 3})

for days in [8]:
    rolls = days @ d
    keys = list(range(-1, days+2))

    fig.add_trace(go.Scatter(
        x=keys,
        y=[rolls.probability(k) for k in keys],
        name=f'days {days}',
        #line_color=COLOR_LIST[i],
        line_shape='hvh',
        fill='tozeroy',
        mode='lines',
        #hoveron='points',
        hovertemplate=
            f'<b>{days} days</b><br>'+
            'encounters %{x:.0f}<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))
    print(rolls.mean(), rolls.median(), rolls.sd())


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-encounter-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-encounter-distribution-small')

2.4 2.0 1.2961481396815724


In [125]:
# Fig 3: Shows the probability of the next encounter occurring after n days.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='rolls',
            range=[0, 11],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,0.41],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.02),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

p_enc = 0.3
p_non = 0.7

rolls = list(range(1, 11))
probs = [p_enc*(p_non**(r-1)) for r in rolls]

fig.add_trace(go.Scatter(
    x=rolls,
    y=probs,
    mode='lines+markers',
    name=f'PDF',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        'rolls %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-rolls-before-encounter-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-rolls-before-encounter-small')

In [126]:
# Fig 4: Compares the probability distribution for a d20 approximate table with that of the original table.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounters',
            range=[-1, 8],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,0.41],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.02),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

travel_days = 8
p_enc = 0.3
p_non = 0.7

# original table
d = Die({0: 7, 1: 3})
rolls = days @ d
keys = list(range(-1, days+2))

fig.add_trace(go.Scatter(
    x=keys,
    y=[rolls.probability(k) for k in keys],
    name=f'original table',
    line_shape='hvh',
    fill='tozeroy',
    mode='lines',
    hovertemplate=
        f'<b>{days} days</b><br>'+
        'encounters %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# alternative encounter table
die_size = 20
d_dict = convert_die(die_size, p_enc)
d = Die(d_dict)
probs = []
last_prob = 0
encounters = list(range(-1, travel_days + 1))
for enc in encounters:
    rolls = (enc+1) @ d
    rolls = rolls.clip(0, 9)
    probs += [rolls.probability(9) - last_prob]
    last_prob = rolls.probability(9)

fig.add_trace(go.Scatter(
    x=np.array(encounters),
    y=np.array(probs),
    name=f'd{die_size} alternative',
    line_shape='hvh',
    fill='tozeroy',
    mode='lines',
    hovertemplate=
        f'<b>d{die_size}</b><br>'+
        'encounters %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-d{die_size}-encounter-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-d{die_size}-encounter-distribution-small')

# save tables
if SAVETABS:
    df = construct_dataframe(d_dict)
    df.to_html(
        f'./tab-alt-d{die_size}.html', 
        columns=[f'1d{die_size}','Days'],
        index=False, 
        classes='wide', 
        border=0,
    )

In [133]:
# Fig 5: Compares the probability distribution for a d20 approximate table with that of the original table.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounters',
            range=[-1, 8],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,0.51],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.02),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

travel_days = 8
p_enc = 0.3
p_non = 0.7

# original table
d = Die({0: 7, 1: 3})
rolls = days @ d
keys = list(range(-1, days+2))

encounters = np.array(keys)
probs = np.array([rolls.probability(k) for k in keys])
fig.add_trace(go.Scatter(
    x=encounters,
    y=probs,
    name=f'original table',
    line_shape='hvh',
    fill='tozeroy',
    mode='lines',
    hovertemplate=
        f'<b>{days} days</b><br>'+
        'encounters %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))
e_mean = np.dot(probs, encounters)
e_var  = np.dot(probs, (encounters - e_mean)**2)
print(f'mean={e_mean:.2f}; std={np.sqrt(e_var):.2f}')

# alternative encounter table
die_size = 20
#d_dict = convert_die(die_size, p_enc)
d_dict = convert_die(die_size, p_enc, n0=2, n1=8)
d = Die(d_dict)
probs = []
last_prob = 0
encounters = list(range(-1, travel_days + 1))
for enc in encounters:
    rolls = (enc+1) @ d
    rolls = rolls.clip(0, 9)
    probs += [rolls.probability(9) - last_prob]
    last_prob = rolls.probability(9)

encounters = np.array(encounters)
probs = np.array(probs)
fig.add_trace(go.Scatter(
    x=encounters,
    y=probs,
    name=f'd{die_size} alternative',
    line_shape='hvh',
    fill='tozeroy',
    mode='lines',
    hovertemplate=
        f'<b>d{die_size}</b><br>'+
        'encounters %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))
e_mean = np.dot(probs, encounters)
e_var  = np.dot(probs, (encounters - e_mean)**2)
print(f'mean={e_mean:.2f}; std={np.sqrt(e_var):.2f}')


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-truncated-d{die_size}-encounter-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-truncated-d{die_size}-encounter-distribution-small')

# save tables
if SAVETABS:
    df = construct_dataframe(d_dict)
    df.to_html(
        f'./tab-truncated-d{die_size}.html', 
        columns=[f'1d{die_size}','Days'],
        index=False, 
        classes='wide', 
        border=0,
    )

mean=2.40; std=1.30
mean=1.92; std=0.75


In [128]:
# Unused figure: Compares the probability distribution for a d6 approximate table with that of the original table.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounters',
            range=[-1, 8],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,0.41],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.02),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

travel_days = 8
p_enc = 0.3
p_non = 0.7

# original table
d = Die({0: 7, 1: 3})
rolls = days @ d
keys = list(range(-1, days+2))

fig.add_trace(go.Scatter(
    x=keys,
    y=[rolls.probability(k) for k in keys],
    name=f'original table',
    #line_color=COLOR_LIST[i],
    line_shape='hvh',
    fill='tozeroy',
    mode='lines',
    #hoveron='points',
    hovertemplate=
        f'<b>{days} days</b><br>'+
        'encounters %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# alternative encounter table
die_size = 6
d_dict = convert_die(die_size, p_enc)
d = Die(d_dict)
probs = []
last_prob = 0
encounters = list(range(-1, travel_days + 1))
for enc in encounters:
    rolls = (enc+1) @ d
    rolls = rolls.clip(0, 9)
    probs += [rolls.probability(9) - last_prob]
    last_prob = rolls.probability(9)

fig.add_trace(go.Scatter(
    x=np.array(encounters),
    y=np.array(probs),
    name=f'd{die_size} alternative',
    line_shape='hvh',
    fill='tozeroy',
    mode='lines',
    hovertemplate=
        f'<b>d{die_size}</b><br>'+
        'encounters %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS and False:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-d{die_size}-encounter-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-d{die_size}-encounter-distribution-small')

# save tables
if SAVETABS and False:
    df = construct_dataframe(d_dict)
    df.to_html(
        f'./tab-alt-d{die_size}.html', 
        columns=[f'1d{die_size}','Days'],
        index=False, 
        classes='wide', 
        border=0,
    )

In [132]:
# Unused figure: Compares the probability distribution for a d12 approximate table with that of the original table.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounters',
            range=[-1, 8],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,0.41],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.02),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

travel_days = 8
p_enc = 0.3
p_non = 0.7

# original table
d = Die({0: 7, 1: 3})
rolls = days @ d
keys = list(range(-1, days+2))

encounters = np.array(keys)
probs = np.array([rolls.probability(k) for k in keys])
fig.add_trace(go.Scatter(
    x=encounters,
    y=probs,
    name=f'original table',
    line_shape='hvh',
    fill='tozeroy',
    mode='lines',
    hovertemplate=
        f'<b>{days} days</b><br>'+
        'encounters %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))
e_mean = np.dot(probs, encounters)
e_var  = np.dot(probs, (encounters - e_mean)**2)
print(f'mean={e_mean:.2f}; std={np.sqrt(e_var):.2f}')

# alternative encounter table
die_size = 12
#d_dict = convert_die(die_size, p_enc)
d_dict = convert_die(die_size, p_enc, n0=2, n1=8)
d = Die(d_dict)
probs = []
last_prob = 0
encounters = list(range(-1, travel_days + 1))
for enc in encounters:
    rolls = (enc+1) @ d
    rolls = rolls.clip(0, 9)
    probs += [rolls.probability(9) - last_prob]
    last_prob = rolls.probability(9)

encounters = np.array(encounters)
probs = np.array(probs)
fig.add_trace(go.Scatter(
    x=encounters,
    y=probs,
    name=f'd{die_size} alternative',
    line_shape='hvh',
    fill='tozeroy',
    mode='lines',
    hovertemplate=
        f'<b>d{die_size}</b><br>'+
        'encounters %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))
e_mean = np.dot(probs, encounters)
e_var  = np.dot(probs, (encounters - e_mean)**2)
print(f'mean={e_mean:.2f}; std={np.sqrt(e_var):.2f}')


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS and False:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-example-d{die_size}-encounter-distribution-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-example-d{die_size}-encounter-distribution-small')

# save tables
if SAVETABS and False:
    df = construct_dataframe(d_dict)
    df.to_html(
        f'./tab-alt-d{die_size}.html', 
        columns=[f'1d{die_size}','Days'],
        index=False, 
        classes='wide', 
        border=0,
    )

mean=2.40; std=1.30
mean=1.97; std=0.75


In [130]:
# Unused figure: Shows the probability of character A's initiative roll being higher than B's, lower than B's, or tying B's
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='rolls',
            range=[0, 10],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,1.01],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.02),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

p_enc = 0.3
p_non = 0.7

rolls = list(range(1, 11))
probs = [p_enc*(p_non**(r-1)) for r in rolls]

fig.add_trace(go.Scatter(
    x=rolls,
    y=probs,
    mode='lines+markers',
    name=f'PDF',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        'rolls %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

rolls = list(range(1, 11))
probs = [1 - p_non**(r) for r in rolls]
fig.add_trace(go.Scatter(
    x=rolls,
    y=probs,
    mode='lines+markers',
    name=f'CDF',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        'rolls %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS and False:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-example-rolls-before-encounter-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-example-rolls-before-encounter-small')